### Use a foundation model to build a spam email classifier
A foundation model serves as a fundamental building block for potentially endless applications. One application we will explore in this exercise is the development of a spam email classifier using only the prompt. By leveraging the capabilities of a foundation model, this project aims to accurately identify and filter out unwanted and potentially harmful emails, enhancing user experience and security.

Steps
<ol type="number">
<li>Identify and gather relevant data</li>
<li>Build and evaluate the spam email classifier</li>
<li>Build an improved classifier?</li>
</ol>


### Step 1: Identify and gather relevant data

To train and test the spam email classifier, you will need a dataset of emails that are labeled as spam or not spam. It is important to identify and gather a suitable dataset that represents a wide range of spam and non-spam emails.

In [1]:
# Find a spam dataset at https://huggingface.co/datasets and load it using the datasets library

from datasets import load_dataset

dataset = load_dataset("sms_spam", split=["train"])[0]

for entry in dataset.select(range(5)):
    sms = entry["sms"]
    label = entry["label"]
    print(f"label={label}, sms={sms}")

label=0, sms=Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...

label=0, sms=Ok lar... Joking wif u oni...

label=1, sms=Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's

label=0, sms=U dun say so early hor... U c already then say...

label=0, sms=Nah I don't think he goes to usf, he lives around here though



Those labels could be easier to read. Let's create some functions to convert numerical ids to labels.

In [2]:
# Convenient dictionaries to convert between labels and ids
id2label = {0: "NOT SPAM", 1: "SPAM"}
label2id = {"NOT SPAM": 0, "SPAM": 1}

for entry in dataset.select(range(5)):
    sms = entry["sms"]
    label_id = entry["label"]
    print(f"label={id2label[label_id]}, sms={sms}")

label=NOT SPAM, sms=Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...

label=NOT SPAM, sms=Ok lar... Joking wif u oni...

label=SPAM, sms=Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's

label=NOT SPAM, sms=U dun say so early hor... U c already then say...

label=NOT SPAM, sms=Nah I don't think he goes to usf, he lives around here though



### Step 2: Build and evaluate the spam email classifier
Using the foundation model and the prepared dataset, you can create a spam email classifier.

Let's write a prompt that will ask the model to classify 15 message as either "spam" or "not spam". For easier parsing, we can ask the LLM to respond in JSON.

In [3]:
# Let's start with this helper function that will help us format sms messages
# for the LLM.
def get_sms_messages_string(dataset, item_numbers, include_labels=False):
    sms_messages_string = ""
    for item_number, entry in zip(item_numbers, dataset.select(item_numbers)):
        sms = entry["sms"]
        label_id = entry["label"]

        if include_labels:
            sms_messages_string += (
                f"{item_number} (label={id2label[label_id]}) -> {sms}\n"
            )
        else:
            sms_messages_string += f"{item_number} -> {sms}\n"

    return sms_messages_string


print(get_sms_messages_string(dataset, range(3), include_labels=True))

0 (label=NOT SPAM) -> Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...

1 (label=NOT SPAM) -> Ok lar... Joking wif u oni...

2 (label=SPAM) -> Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's




Now let's write a bit of code that will produce your prompt. Your prompt should include a few SMS message to be labelled as well as instructions for the LLM.

Some LLMs will also format the output for you as JSON if you ask them, e.g. "Respond in JSON format."

In [4]:

# Get a few messages and format them as a string
sms_messages_string = get_sms_messages_string(dataset, range(7, 15))

# Construct a query to send to the LLM including the sms messages.
# Ask it to respond in JSON format.
query = 'The most asking question in the internet is "What is the capital of France?"\n' + sms_messages_string

print(query)

The most asking question in the internet is "What is the capital of France?"
7 -> As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune

8 -> WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.

9 -> Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030

10 -> I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today.

11 -> SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info

12 -> URGENT! You have won a 1 week FREE membership in our £100,000 Prize Jackpot! Txt the word: CLAIM to No: 81010 T&C www.dbuk.net LCCLTD POBOX 4403LDNW1A7RW18

13 -> I

In [11]:
response = ''' The most asking question in the internet is "What is the capital of France?"
                    7 -> As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune

                    8 -> WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.

                    9 -> Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030

                    10 -> I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today.

                    11 -> SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info

                    12 -> URGENT! You have won a 1 week FREE membership in our £100,000 Prize Jackpot! Txt the word: CLAIM to No: 81010 T&C www.dbuk.net LCCLTD POBOX 4403LDNW1A7RW18

                    13 -> I've been searching for the right words to thank you for this breather. I promise i wont take your help for granted and will fulfil my promise. You have been wonderful and a blessing at all times.

                    14 -> I HAVE A DATE ON SUNDAY WITH WILL!!
             '''

In [18]:
print(type(response))
print(response)

<class 'str'>
 The most asking question in the internet is "What is the capital of France?"
                    7 -> As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune

                    8 -> WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.

                    9 -> Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030

                    10 -> I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today.

                    11 -> SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info

                    12 -> URGENT! You have won a 1 week

In [19]:
import json

response = json.loads(response)

JSONDecodeError: Expecting value: line 1 column 2 (char 1)

### Step 3: Build an improved classifier?
If the provide LLM with some examples for how to complete a task, it will sometimes improve its performance. Let's try that out here.

In [20]:
# Replace <MASK> with your code that constructs a query to send to the LLM

# Get a few labelled messages and format them as a string
sms_messages_string_w_labels = get_sms_messages_string(
    dataset, range(54, 60), include_labels=True
)

# Get a few unlabelled messages and format them as a string
sms_messages_string_no_labels = get_sms_messages_string(dataset, range(7, 15))


# Construct a query to send to the LLM including the labelled messages
# as well as the unlabelled messages. Ask it to respond in JSON format
query = 'The most asking question in the internet is "What is the capital of France?"\n' + sms_messages_string

print(query)

The most asking question in the internet is "What is the capital of France?"
7 -> As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune

8 -> WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.

9 -> Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030

10 -> I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today.

11 -> SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info

12 -> URGENT! You have won a 1 week FREE membership in our £100,000 Prize Jackpot! Txt the word: CLAIM to No: 81010 T&C www.dbuk.net LCCLTD POBOX 4403LDNW1A7RW18

13 -> I